<a href="https://colab.research.google.com/github/DojunPark/Machine_Translation/blob/master/14_diversification_of_tokenizing_method.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Drive mount

In [53]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!nvidia-smi    # tesla v100

Sun Nov  1 05:50:25 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.32.00    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    25W / 300W |      0MiB / 16130MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [5]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

Your runtime has 27.4 gigabytes of available RAM



# install hgtk package

In [5]:
!pip install hgtk

  Created wheel for hgtk: filename=hgtk-0.1.3-py2.py3-none-any.whl size=6688 sha256=4ccc2816a29ceb6ef62a172f68dbae24d872d75dd8886d6c5da249a2993db88c
  Stored in directory: /root/.cache/pip/wheels/73/72/06/6065a57fe68264f35d7e52e37f56831eb3e9ec75656880de20
Successfully built hgtk


In [12]:
import hgtk

tokens = hgtk.letter.decompose('값')
# tokens = list(tokens)
print(tokens)
print(type(tokens))

('ㄱ', 'ㅏ', 'ㅄ')
<class 'tuple'>


In [13]:
hgtk.letter.compose('ㄱ', 'ㅏ', 'ㅄ')

'값'

In [14]:
hgtk.letter.compose('ㄱ', 'ㅏ', 'ㅁ')

'감'

In [17]:
txt = '안녕하세요 제 이름은 박도준입니다. 영어로는 Dojun이라고 씁니다.'
result = hgtk.text.decompose(txt)
print(result)

ㅇㅏㄴᴥㄴㅕㅇᴥㅎㅏᴥㅅㅔᴥㅇㅛᴥ ㅈㅔᴥ ㅇㅣᴥㄹㅡㅁᴥㅇㅡㄴᴥ ㅂㅏㄱᴥㄷㅗᴥㅈㅜㄴᴥㅇㅣㅂᴥㄴㅣᴥㄷㅏᴥ. ㅇㅕㅇᴥㅇㅓᴥㄹㅗᴥㄴㅡㄴᴥ Dojunㅇㅣᴥㄹㅏᴥㄱㅗᴥ ㅆㅡㅂᴥㄴㅣᴥㄷㅏᴥ.


In [19]:
result2 = hgtk.text.compose(result)
print(result2)

안녕하세요 제 이름은 박도준입니다. 영어로는 Dojun이라고 씁니다.


# SencencePiece Tokenizer

In [51]:
!pip install sentencepiece

     |████████████████████████████████| 1.1MB 4.9MB/s 


In [76]:
!cd /content/drive/My\ Drive/

In [77]:
pwd

'/content/drive/My Drive'

- 방법1

In [80]:
templates= '--input={} \
--pad_id={} \
--bos_id={} \
--eos_id={} \
--unk_id={} \
--model_prefix={} \
--vocab_size={} \
--character_coverage={} \
--model_type={}'

train_input_file = "train.csv"
pad_id=0     #<pad> token을 0으로 설정
vocab_size = 20000 # vocab 사이즈
prefix = 'm' # 저장될 tokenizer 모델에 붙는 이름
bos_id=1     #<start> token을 1으로 설정
eos_id=2     #<end> token을 2으로 설정
unk_id=3     #<unknown> token을 3으로 설정
character_coverage = 1.0   # to reduce character set 
model_type ='word'         # Choose from unigram (default), bpe, char, or word

cmd = templates.format(train_input_file,
                pad_id,
                bos_id,
                eos_id,
                unk_id,
                prefix,
                vocab_size,
                character_coverage,
                model_type)

In [81]:
spm.SentencePieceTrainer.Train(cmd)

- 방법2

In [83]:
import sentencepiece as spm

spm.SentencePieceTrainer.train('--input=train.csv --model_prefix=m --vocab_size=20000')

- load

In [84]:
sp = spm.SentencePieceProcessor()
sp.load('m.model')

True

In [114]:
# encode
print(sp.encode_as_pieces('This is a test, 이것은 테스트입니다.'))
print(sp.encode_as_ids('This is a test, 이것은 테스트입니다.'))

# decode
print(sp.decode_pieces(['▁', 'This', '▁is', '▁a', '▁test', ',', '▁이것', '은', '▁테스트', '입니다', '.']))
print(sp.decode_ids([5, 375, 21, 12, 1430, 3, 12643, 26, 11642, 2348, 24]))

['▁', 'This', '▁is', '▁a', '▁test', ',', '▁이것', '은', '▁테스트', '입니다', '.']
[5, 375, 21, 12, 1430, 3, 12643, 26, 11642, 2348, 24]
This is a test, 이것은 테스트입니다.
This is a test, 이것은 테스트입니다.


# 자모 분리 Tokenizer

In [101]:
import hgtk

def seperate_cv(sentence):
    decompossed = hgtk.text.decompose(sentence)
    decompossed = decompossed.replace('ᴥ', '')

    result = list()
    for cv in decompossed:
        result.append(cv)

    return result

# 음절 분리 / 알파벳 분리

In [110]:
def seperate_char(sentence):
    sent = sentence.lower()
    result = list()
    for char in sent:
        result.append(char)

    return result

In [112]:
sentence1 = '각 형태소 분석기는 성능과 속도의 차이가 있기에, 본인의 사용 목적에 알맞게 선택하도록 합니다.'
sentence2 ="I am actively looking for Ph.D. students. and you are a Ph.D. student."

print(seperate_char(sentence1))
print(seperate_char(sentence2))

['각', ' ', '형', '태', '소', ' ', '분', '석', '기', '는', ' ', '성', '능', '과', ' ', '속', '도', '의', ' ', '차', '이', '가', ' ', '있', '기', '에', ',', ' ', '본', '인', '의', ' ', '사', '용', ' ', '목', '적', '에', ' ', '알', '맞', '게', ' ', '선', '택', '하', '도', '록', ' ', '합', '니', '다', '.']
['i', ' ', 'a', 'm', ' ', 'a', 'c', 't', 'i', 'v', 'e', 'l', 'y', ' ', 'l', 'o', 'o', 'k', 'i', 'n', 'g', ' ', 'f', 'o', 'r', ' ', 'p', 'h', '.', 'd', '.', ' ', 's', 't', 'u', 'd', 'e', 'n', 't', 's', '.', ' ', 'a', 'n', 'd', ' ', 'y', 'o', 'u', ' ', 'a', 'r', 'e', ' ', 'a', ' ', 'p', 'h', '.', 'd', '.', ' ', 's', 't', 'u', 'd', 'e', 'n', 't', '.']


# 어절 분리
- nltk 라이브러리의 word_tokenize() 메소드를 사용하도록 한다

In [116]:
sentence = '각 형태소 분석기는 성능과 속도의 차이가 있기에, 본인의 사용 목적에 알맞게 선택하도록 합니다.'

import nltk
sent = nltk.word_tokenize(sentence)
print(sent)

['각', '형태소', '분석기는', '성능과', '속도의', '차이가', '있기에', ',', '본인의', '사용', '목적에', '알맞게', '선택하도록', '합니다', '.']
